# Solution to Exercise 6 - Process Tasks in Order of Completion

**GOAL:** The goal of this exercise is to show how to use `ray.wait` to process tasks in the order that they finish.

See the documentation for ray.wait at http://ray.readthedocs.io/en/latest/api.html#waiting-for-a-subset-of-tasks-to-finish.

The code below runs 10 tasks and retrieves the results in the order that the tasks were launched. However, since each task takes a random amount of time to finish, we could instead process the tasks in the order that they finish.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

In [2]:
ray.init(num_cpus=5, redirect_output=True)

Waiting for redis server at 127.0.0.1:14568 to respond...
Waiting for redis server at 127.0.0.1:58432 to respond...
Starting local scheduler with 5 CPUs, 0 GPUs

View the web UI at http://localhost:8898/notebooks/ray_ui22125.ipynb?token=15e82bfc4d2507595d3a038d15eda02d51652612b05a2c7e



{'local_scheduler_socket_names': ['/tmp/scheduler66481052'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store79155088', manager_name='/tmp/plasma_manager62186845', manager_port=41265)],
 'redis_address': '127.0.0.1:14568',
 'webui_url': 'http://localhost:8898/notebooks/ray_ui22125.ipynb?token=15e82bfc4d2507595d3a038d15eda02d51652612b05a2c7e'}

In [3]:
@ray.remote
def f():
    time.sleep(np.random.uniform(0, 5))
    return time.time()

**EXERCISE:** Change the code below to use `ray.wait` to get the results of the tasks in the order that they complete.

**NOTE:** It would be a simple modification to maintain a pool of 10 experiments and to start a new experiment whenever one finishes.

In [4]:
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

result_ids = [f.remote() for _ in range(10)]

# Get the results.
results = []
while len(result_ids):
    ready_id, remaining_ids = ray.wait(result_ids, num_returns= 1) # Get the Object ID which is ready in the object store
    result = ray.get(ready_id[0])
    results.append(result)
    result_ids = remaining_ids # Reset the result_ids to contain those IDs which are yet to be fetched 
    print('Processing result which finished after {} seconds.'
          .format(result - start_time))

end_time = time.time()
duration = end_time - start_time

Processing result which finished after 1.5506346225738525 seconds.
Processing result which finished after 3.3468384742736816 seconds.
Processing result which finished after 3.8243601322174072 seconds.
Processing result which finished after 4.136775016784668 seconds.
Processing result which finished after 4.304465293884277 seconds.
Processing result which finished after 5.513854503631592 seconds.
Processing result which finished after 5.573364734649658 seconds.
Processing result which finished after 5.708314657211304 seconds.
Processing result which finished after 7.56557297706604 seconds.
Processing result which finished after 8.792317390441895 seconds.


**VERIFY:** Run some checks to verify that the changes you made to the code were correct. Some of the checks should fail when you initially run the cells. After completing the exercises, the checks should pass.

In [5]:
assert results == sorted(results), ('The results were not processed in the '
                                    'order that they finished.')

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 8.794857740402222 seconds.
